In [1]:
data = [1,2,3,4,5]
distData = sc.parallelize(data)
Sum = distData.map(lambda x: x**2)
print(Sum.take(10))
print(Sum.reduce(lambda x, y: x+y))

55
[1, 4, 9, 16, 25]

In [2]:
import requests
s = requests.get("https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt")
with open("/dbfs/test/alice.txt", "w") as f:
  f.write(s.text)

Out[30]: True

In [3]:
file = sc.textFile("dbfs:/test/alice.txt")
from string import punctuation
translate = "".maketrans(punctuation, " "*len(punctuation))
lines = file.flatMap(lambda x: x.split("\n")).filter(lambda x: len(x)>0).map(lambda x: x.translate(translate))
wordcount = lines.flatMap(lambda x: x.split()).map(lambda x: (x.lower(), 1)).reduceByKey(lambda x, y: x+y).sortByKey(True)
print(wordcount.take(100))
# for word in wordcount.collect()[:10]:
#   print(word)

[('0', 1), ('3', 1), ('a', 632), ('abide', 1), ('able', 1), ('about', 94), ('above', 3), ('absence', 1), ('absurd', 2), ('acceptance', 1), ('accident', 2), ('accidentally', 1), ('account', 1), ('accounting', 1), ('accounts', 1), ('accusation', 1), ('accustomed', 1), ('ache', 1), ('across', 5), ('act', 1), ('actually', 1), ('ada', 1), ('added', 23), ('adding', 1), ('addressed', 2), ('addressing', 1), ('adjourn', 1), ('adoption', 1), ('advance', 3), ('advantage', 3), ('adventures', 8), ('advice', 2), ('advisable', 2), ('advise', 1), ('affair', 1), ('affectionately', 1), ('afford', 1), ('afore', 1), ('afraid', 12), ('after', 43), ('afterwards', 2), ('again', 83), ('against', 9), ('age', 4), ('ago', 2), ('agony', 1), ('agree', 2), ('ah', 5), ('ahem', 1), ('air', 15), ('airs', 1), ('alarm', 2), ('alarmed', 1), ('alas', 4), ('alice', 399), ('alive', 3), ('all', 182), ('allow', 3), ('almost', 6), ('alone', 4), ('along', 6), ('aloud', 5), ('already', 2), ('also', 2), ('altered', 1), ('alternately', 1), ('altogether', 5), ('always', 13), ('am', 16), ('ambition', 1), ('among', 12), ('an', 57), ('ancient', 1), ('and', 872), ('anger', 2), ('angrily', 9), ('angry', 5), ('animal', 2), ('animals', 4), ('ann', 4), ('annoy', 1), ('annoyed', 1), ('another', 22), ('answer', 9), ('answered', 4), ('answers', 1), ('antipathies', 1), ('anxious', 3), ('anxiously', 14), ('any', 39), ('anything', 20), ('anywhere', 1), ('appealed', 1), ('appear', 2), ('appearance', 1), ('appeared', 8), ('appearing', 1), ('applause', 1), ('apple', 1), ('apples', 2)]

In [4]:
from pyspark import sql
d1 = sql.Row(dno="132", name="science")
d2 = sql.Row(dno="235", name="math")
d3 = sql.Row(dno="564", name="social")
d = [d1, d2, d3]
df1 = spark.createDataFrame(d)
df1.show()

+---+-------+
dno| name|
+---+-------+
132|science|
235| math|
564| social|
+---+-------+

In [5]:
Employee = sql.Row("ssn", "fname", "lname", "email", "salary", "dno")
e1 = Employee("888-999", "rohan", "kumar", "rk@gmail.com", 6780000, 132)
e2 = Employee("898-999", "popat", "lal", "pl@gmail.com", 6000000, 235)
e = [e1, e2]
df2 = spark.createDataFrame(e)
df2.show()

+-------+-----+-----+------------+-------+---+
 ssn|fname|lname| email| salary|dno|
+-------+-----+-----+------------+-------+---+
888-999|rohan|kumar|rk@gmail.com|6780000|132|
898-999|popat| lal|pl@gmail.com|6000000|235|
+-------+-----+-----+------------+-------+---+

In [6]:
df2.join(df1, df1.dno == df2.dno, "inner").show()
df1.registerTempTable("Employee")
df2.registerTempTable("Department")
result = sqlContext.sql("select * from Employee as e join Department as d on e.dno=d.dno")
result.show()

+-------+-----+-----+------------+-------+---+---+-------+
 ssn|fname|lname| email| salary|dno|dno| name|
+-------+-----+-----+------------+-------+---+---+-------+
898-999|popat| lal|pl@gmail.com|6000000|235|235| math|
888-999|rohan|kumar|rk@gmail.com|6780000|132|132|science|
+-------+-----+-----+------------+-------+---+---+-------+

+---+-------+-------+-----+-----+------------+-------+---+
dno| name| ssn|fname|lname| email| salary|dno|
+---+-------+-------+-----+-----+------------+-------+---+
235| math|898-999|popat| lal|pl@gmail.com|6000000|235|
132|science|888-999|rohan|kumar|rk@gmail.com|6780000|132|
+---+-------+-------+-----+-----+------------+-------+---+